# Grounded Agents: E-commerce Agent with Ontology

This notebook demonstrates how to build an AI agent grounded in a knowledge graph using GraphForge.

**What is agent grounding?**
Grounding uses a knowledge graph to:
- Structure domain knowledge (products, orders, inventory)
- Annotate tools with capabilities
- Enable semantic tool selection
- Provide reasoning paths for the agent

**Why GraphForge?**
- Embedded (no server setup)
- Python-native (seamless integration)
- openCypher compatible (expressive queries)
- Zero configuration (perfect for agent development)

## Setup

In [ ]:
from graphforge import GraphForge
import json

# Create embedded graph database
gf = GraphForge()
print("✅ GraphForge initialized (embedded, zero-config)")

## Step 1: Define Domain Ontology

Create a structured model of the e-commerce domain with classes, properties, and relationships.

In [ ]:
# Define domain classes
gf.execute("""
    CREATE (:Class {name: 'Entity', description: 'Base class for all domain entities'})
    CREATE (:Class {name: 'Product', description: 'Physical or digital product'})
    CREATE (:Class {name: 'Electronics', description: 'Electronic products'})
    CREATE (:Class {name: 'Clothing', description: 'Apparel and accessories'})
    CREATE (:Class {name: 'Inventory', description: 'Stock tracking system'})
    CREATE (:Class {name: 'Order', description: 'Customer purchase order'})
    CREATE (:Class {name: 'Customer', description: 'User account'})
""")

# Define class hierarchy (IS_A relationships)
gf.execute("""
    MATCH (product:Class {name: 'Product'}), (entity:Class {name: 'Entity'})
    CREATE (product)-[:IS_A]->(entity)
    
    MATCH (electronics:Class {name: 'Electronics'}), (product:Class {name: 'Product'})
    CREATE (electronics)-[:IS_A]->(product)
    
    MATCH (clothing:Class {name: 'Clothing'}), (product:Class {name: 'Product'})
    CREATE (clothing)-[:IS_A]->(product)
    
    MATCH (inventory:Class {name: 'Inventory'}), (entity:Class {name: 'Entity'})
    CREATE (inventory)-[:IS_A]->(entity)
    
    MATCH (order:Class {name: 'Order'}), (entity:Class {name: 'Entity'})
    CREATE (order)-[:IS_A]->(entity)
    
    MATCH (customer:Class {name: 'Customer'}), (entity:Class {name: 'Entity'})
    CREATE (customer)-[:IS_A]->(entity)
""")

# Define properties for each class
gf.execute("""
    CREATE (:Property {name: 'id', type: 'string', class: 'Product', description: 'Unique product identifier'})
    CREATE (:Property {name: 'name', type: 'string', class: 'Product', description: 'Product name'})
    CREATE (:Property {name: 'price', type: 'float', class: 'Product', description: 'Price in USD'})
    CREATE (:Property {name: 'category', type: 'string', class: 'Product', description: 'Product category'})
    CREATE (:Property {name: 'brand', type: 'string', class: 'Product', description: 'Manufacturer brand'})
    
    CREATE (:Property {name: 'product_id', type: 'string', class: 'Inventory', description: 'Product being tracked'})
    CREATE (:Property {name: 'quantity', type: 'int', class: 'Inventory', description: 'Current stock level'})
    CREATE (:Property {name: 'warehouse', type: 'string', class: 'Inventory', description: 'Storage location'})
    
    CREATE (:Property {name: 'order_id', type: 'string', class: 'Order', description: 'Unique order identifier'})
    CREATE (:Property {name: 'customer_id', type: 'string', class: 'Order', description: 'Customer who placed order'})
    CREATE (:Property {name: 'total', type: 'float', class: 'Order', description: 'Order total amount'})
    CREATE (:Property {name: 'status', type: 'string', class: 'Order', description: 'Order status'})
""")

print("✅ Domain ontology created")
print("   - 7 classes (Entity, Product, Electronics, Clothing, Inventory, Order, Customer)")
print("   - 6 IS_A relationships (class hierarchy)")
print("   - 13 properties defined")

## Step 2: Annotate Tools with Capabilities

Define available tools and link them to domain concepts.

In [ ]:
# Tool 1: Search Products
gf.execute("""
    CREATE (t:Tool {
        name: 'search_products',
        description: 'Search for products by keyword, category, or brand',
        returns: 'list[Product]',
        endpoint: 'api.products.search'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'search_products'})
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'query',
        type: 'string',
        required: true,
        description: 'Search query (keyword, category, or brand)'
    })
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'limit',
        type: 'int',
        required: false,
        description: 'Maximum number of results (default: 10)'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'search_products'}), (c:Class {name: 'Product'})
    CREATE (t)-[:OPERATES_ON]->(c)
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'search'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'find_products'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'browse_catalog'})
""")

# Tool 2: Check Inventory
gf.execute("""
    CREATE (t:Tool {
        name: 'check_inventory',
        description: 'Check current stock levels for a product',
        returns: 'int',
        endpoint: 'api.inventory.check'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'check_inventory'})
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'product_id',
        type: 'string',
        required: true,
        description: 'Product identifier'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'check_inventory'}),
          (inv:Class {name: 'Inventory'}),
          (prod:Class {name: 'Product'})
    CREATE (t)-[:OPERATES_ON]->(inv)
    CREATE (t)-[:REQUIRES]->(prod)
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'check_stock'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'verify_availability'})
""")

# Tool 3: Get Price
gf.execute("""
    CREATE (t:Tool {
        name: 'get_price',
        description: 'Get current price for a product',
        returns: 'float',
        endpoint: 'api.products.price'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'get_price'})
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'product_id',
        type: 'string',
        required: true,
        description: 'Product identifier'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'get_price'}), (c:Class {name: 'Product'})
    CREATE (t)-[:OPERATES_ON]->(c)
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'get_pricing'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'check_cost'})
""")

# Tool 4: Place Order
gf.execute("""
    CREATE (t:Tool {
        name: 'place_order',
        description: 'Create a new order for products',
        returns: 'Order',
        endpoint: 'api.orders.create'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'place_order'})
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'customer_id',
        type: 'string',
        required: true,
        description: 'Customer identifier'
    })
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'items',
        type: 'list[dict]',
        required: true,
        description: 'List of {product_id, quantity} dicts'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'place_order'}),
          (order:Class {name: 'Order'}),
          (customer:Class {name: 'Customer'}),
          (product:Class {name: 'Product'})
    CREATE (t)-[:OPERATES_ON]->(order)
    CREATE (t)-[:REQUIRES]->(customer)
    CREATE (t)-[:REQUIRES]->(product)
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'purchase'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'create_order'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'checkout'})
""")

# Tool 5: Track Shipment
gf.execute("""
    CREATE (t:Tool {
        name: 'track_shipment',
        description: 'Get shipping status and tracking information for an order',
        returns: 'dict',
        endpoint: 'api.orders.tracking'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'track_shipment'})
    CREATE (t)-[:HAS_PARAMETER]->(:Parameter {
        name: 'order_id',
        type: 'string',
        required: true,
        description: 'Order identifier'
    })
""")

gf.execute("""
    MATCH (t:Tool {name: 'track_shipment'}), (c:Class {name: 'Order'})
    CREATE (t)-[:OPERATES_ON]->(c)
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'track_order'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'check_delivery'})
    CREATE (t)-[:CAN_DO]->(:Capability {name: 'shipping_status'})
""")

print("✅ Tools annotated with capabilities")
print("   - 5 tools defined (search_products, check_inventory, get_price, place_order, track_shipment)")
print("   - 8 parameters across tools")
print("   - 13 capabilities mapped")
print("   - Tools linked to domain concepts via OPERATES_ON and REQUIRES relationships")

## Step 3: Visualize the Ontology

Let's see the structure we've created.

In [ ]:
# Get class hierarchy
hierarchy = gf.execute("""
    MATCH (child:Class)-[:IS_A]->(parent:Class)
    RETURN child.name AS child, parent.name AS parent
    ORDER BY parent, child
""")

print("Class Hierarchy:")
for r in hierarchy:
    print(f"  {r['child']} → IS_A → {r['parent']}")

# Get tool-to-concept mappings
mappings = gf.execute("""
    MATCH (t:Tool)-[r:OPERATES_ON|REQUIRES]->(c:Class)
    RETURN t.name AS tool, type(r) AS relationship, c.name AS concept
    ORDER BY tool, concept
""")

print("\nTool-Concept Mappings:")
current_tool = None
for r in mappings:
    if r['tool'] != current_tool:
        print(f"\n  {r['tool']}:")
        current_tool = r['tool']
    print(f"    {r['relationship']} → {r['concept']}")

## Step 4: Agent Query Patterns

Show how an agent queries the ontology for tool selection.

In [ ]:
def find_tools_by_capability(capability_keyword):
    """Find tools that match a capability keyword."""
    query = """
    MATCH (t:Tool)-[:CAN_DO]->(c:Capability)
    WHERE c.name CONTAINS $keyword
    RETURN t.name AS tool,
           t.description AS description,
           collect(c.name) AS capabilities
    """
    results = gf.execute(query, {'keyword': capability_keyword})
    return [dict(r) for r in results]

# Example: Agent intent is "check product availability"
print("Query: Find tools for 'stock' or 'availability'")
tools = find_tools_by_capability('stock')
for tool in tools:
    print(f"\n  Tool: {tool['tool']}")
    print(f"  Description: {tool['description']}")
    print(f"  Capabilities: {', '.join(tool['capabilities'])}")

In [ ]:
def find_tools_for_entity(entity_class):
    """Find all tools that work with an entity or its parent classes."""
    query = """
    MATCH (c:Class {name: $entity_class})-[:IS_A*0..]->(parent:Class)
    MATCH (t:Tool)-[:OPERATES_ON]->(parent)
    RETURN DISTINCT t.name AS tool,
           t.description AS description,
           parent.name AS operates_on
    ORDER BY tool
    """
    results = gf.execute(query, {'entity_class': entity_class})
    return [dict(r) for r in results]

# Example: Agent is working with Products
print("\nQuery: Find tools that work with 'Product' entities")
tools = find_tools_for_entity('Product')
for tool in tools:
    print(f"\n  Tool: {tool['tool']}")
    print(f"  Description: {tool['description']}")
    print(f"  Operates on: {tool['operates_on']}")

In [ ]:
def get_tool_metadata(tool_name):
    """Get complete tool definition including parameters."""
    query = """
    MATCH (t:Tool {name: $tool_name})
    OPTIONAL MATCH (t)-[:HAS_PARAMETER]->(p:Parameter)
    OPTIONAL MATCH (t)-[:OPERATES_ON]->(c:Class)
    OPTIONAL MATCH (t)-[:CAN_DO]->(cap:Capability)
    RETURN t.name AS name,
           t.description AS description,
           t.endpoint AS endpoint,
           t.returns AS returns,
           collect(DISTINCT {name: p.name, type: p.type, required: p.required, description: p.description}) AS parameters,
           collect(DISTINCT c.name) AS operates_on,
           collect(DISTINCT cap.name) AS capabilities
    """
    result = gf.execute(query, {'tool_name': tool_name})
    return dict(result[0]) if result else None

# Example: Get full metadata for selected tool
print("\nQuery: Get metadata for 'check_inventory' tool")
metadata = get_tool_metadata('check_inventory')
print(json.dumps(metadata, indent=2))

## Step 5: Multi-Step Reasoning

Find tool chains for complex workflows.

In [ ]:
def find_tool_chains():
    """Find tools that can be composed (output of one feeds input of another)."""
    # This is a simplified version - in practice you'd model PRODUCES relationships
    # For this demo, we'll infer chains from shared concepts
    query = """
    MATCH (t1:Tool)-[:OPERATES_ON]->(c:Class)<-[:REQUIRES]-(t2:Tool)
    RETURN t1.name AS first_tool,
           c.name AS intermediate_concept,
           t2.name AS next_tool,
           t1.description AS first_desc,
           t2.description AS next_desc
    """
    results = gf.execute(query)
    return [dict(r) for r in results]

# Example: Find composable tool chains
print("Query: Find tool chains (composable workflows)")
chains = find_tool_chains()
for chain in chains:
    print(f"\n  Pipeline: {chain['first_tool']} → {chain['intermediate_concept']} → {chain['next_tool']}")
    print(f"    1. {chain['first_desc']}")
    print(f"    2. {chain['next_desc']}")

## Step 6: Simple Agent Demo

Simulate an agent using the ontology for grounded reasoning.

In [ ]:
class OntologyGroundedAgent:
    """Simple agent that uses ontology for tool grounding."""
    
    def __init__(self, gf):
        self.gf = gf
    
    def handle_query(self, user_query, intent_keyword):
        """Handle user query with ontology-grounded tool selection."""
        print(f"\n{'='*60}")
        print(f"User Query: {user_query}")
        print(f"{'='*60}")
        
        # Step 1: Query ontology for relevant tools
        print(f"\n[Agent] Querying ontology for tools related to '{intent_keyword}'...")
        query = """
        MATCH (t:Tool)-[:CAN_DO]->(c:Capability)
        WHERE c.name CONTAINS $intent
        RETURN t.name AS tool, t.description AS description
        LIMIT 1
        """
        results = self.gf.execute(query, {'intent': intent_keyword})
        
        if not results:
            print("[Agent] No tools found for this intent.")
            return
        
        selected_tool = results[0]['tool']
        description = results[0]['description']
        
        print(f"[Agent] Found tool: {selected_tool}")
        print(f"[Agent] Description: {description}")
        
        # Step 2: Get tool metadata
        print(f"\n[Agent] Retrieving tool metadata...")
        metadata = get_tool_metadata(selected_tool)
        
        print(f"[Agent] Parameters required:")
        for param in metadata['parameters']:
            if param['name']:  # Filter out empty params from collect
                req = "required" if param['required'] else "optional"
                print(f"  - {param['name']} ({param['type']}, {req}): {param['description']}")
        
        # Step 3: Simulate execution
        print(f"\n[Agent] Executing {selected_tool}...")
        print(f"[Agent] Endpoint: {metadata['endpoint']}")
        print(f"[Agent] Returns: {metadata['returns']}")
        print(f"\n[Agent] ✅ Task completed using grounded tool selection")

# Create agent
agent = OntologyGroundedAgent(gf)

# Simulate agent interactions
agent.handle_query(
    "Is product XYZ in stock?",
    "stock"
)

agent.handle_query(
    "I want to buy some electronics",
    "search"
)

agent.handle_query(
    "Where is my order?",
    "track"
)

## Step 7: Contextual Tool Selection

Select tools based on entities mentioned in the conversation.

In [ ]:
def select_contextual_tools(conversation_entities):
    """Find tools relevant to entities mentioned in conversation."""
    query = """
    MATCH (c:Class)<-[:IS_A*0..]-(entity:Class)
    WHERE entity.name IN $entities
    MATCH (t:Tool)-[:OPERATES_ON|REQUIRES]->(c)
    RETURN DISTINCT t.name AS tool,
           t.description AS description,
           entity.name AS relevant_to
    ORDER BY t.name
    """
    results = gf.execute(query, {'entities': conversation_entities})
    return [dict(r) for r in results]

# Example: Conversation mentions "Product" and "Order"
print("Query: Tools relevant to conversation about 'Product' and 'Order'")
tools = select_contextual_tools(['Product', 'Order'])

for tool in tools:
    print(f"\n  Tool: {tool['tool']}")
    print(f"  Relevant to: {tool['relevant_to']}")
    print(f"  Description: {tool['description']}")

## Summary

This notebook demonstrated:

1. **Domain Ontology**: Structured e-commerce knowledge (classes, properties, relationships)
2. **Tool Annotations**: Linked tools to domain concepts with capabilities
3. **Semantic Queries**: Found tools by intent, not just keywords
4. **Tool Metadata**: Retrieved complete tool signatures for execution
5. **Multi-Step Reasoning**: Identified tool chains for workflows
6. **Agent Simulation**: Used ontology for grounded tool selection
7. **Contextual Selection**: Adapted tool choices to conversation context

### Why GraphForge?

- ✅ **Embedded**: No server, no configuration, zero deployment overhead
- ✅ **Python-native**: Seamless integration with agent code
- ✅ **Expressive**: Full openCypher query power
- ✅ **Fast**: In-process queries, no network latency
- ✅ **Portable**: Works in notebooks, scripts, containers, serverless

### Next Steps

1. **Integrate with LLM**: Connect to GPT-4, Claude, or other LLMs
2. **Add real tools**: Implement actual API calls
3. **Expand ontology**: Add more domain classes and relationships
4. **Permission model**: Add user roles and access control
5. **Learn from usage**: Track tool effectiveness, refine annotations

### Resources

- [GraphForge Documentation](https://github.com/DecisionNerd/graphforge)
- [Agent Grounding Guide](../../docs/use-cases/agent-grounding.md)
- [openCypher Tutorial](../../docs/tutorial.md)
